In [5]:
import math
import pandas as pd
import re
import folium
import pygal
from pygal.style import Style
import ipywidgets as widgets
from IPython.display import display, HTML

# Load the data
data = pd.read_csv('N2K Recording 1.n2kdecoded_Dummy_new.csv')

# Convert 'Timestamp' to datetime
data['Timestamp'] = pd.to_datetime(data['Timestamp'])

# Filter and convert 'Course Over Ground' from radians to degrees
cog_data = data[data['Name'] == 'Course Over Ground']
cog_data['Value'] = cog_data['Value'].astype(float) * (180 / math.pi)

heading_data = data[data['Name'] == 'Heading']
heading_data['Value'] = heading_data['Value'].astype(float) * (180 / math.pi)

# Get the most recent timestamp
most_recent_time = data['Timestamp'].max()

# Get all unique sources
all_sources = data['Source'].unique().tolist()

# Function to convert latitude/longitude to decimal degrees
def convert_to_decimal_degrees(coord):
    if isinstance(coord, str):
        pattern = r"(\d+)° (\d+\.\d+)' ([NSEW])"
        match = re.match(pattern, coord)
        if match:
            degrees, minutes, direction = match.groups()
            decimal_degrees = float(degrees) + float(minutes) / 60
            if direction in ['S', 'W']:
                decimal_degrees *= -1
            return decimal_degrees
    return 0.0

# Function to create a compass-like plot with triangles, reduced by 30%
def create_compass(angle):
    scale_factor = 0.7  # Scale factor to reduce size by 30%
    
    # Adjusted dimensions for the SVG canvas
    canvas_size = int(500 * scale_factor)
    center = int(200 * scale_factor)
    outer_radius = int(200 * scale_factor)
    inner_radius = int(10 * scale_factor)
    triangle_offset = int(20 * scale_factor)
    
    directions = ['E', 'NE', 'N', 'NW', 'W', 'SW', 'S', 'SE']
    angles = [0, 45, 90, 135, 180, 225, 270, 315]
    
    custom_svg = f"""
    <svg xmlns="http://www.w3.org/2000/svg" width="{canvas_size}" height="{canvas_size}">
    """
    
    # Draw the border circle
    custom_svg += f"""
    <circle cx="{center}" cy="{center}" r="{outer_radius}" stroke="black" stroke-width="2" fill="none" />
    """
    
    # Draw the small circle in the center
    custom_svg += f"""
    <circle cx="{center}" cy="{center}" r="{inner_radius}" fill="black" />
    """
    
    # Draw the cardinal and intercardinal triangles
    for direction, angle_deg in zip(directions, angles):
        x1 = center + outer_radius * math.cos(math.radians(angle_deg))
        y1 = center - outer_radius * math.sin(math.radians(angle_deg))
        x2 = center + triangle_offset * math.cos(math.radians(angle_deg - 10))
        y2 = center - triangle_offset * math.sin(math.radians(angle_deg - 10))
        x3 = center + triangle_offset * math.cos(math.radians(angle_deg + 10))
        y3 = center - triangle_offset * math.sin(math.radians(angle_deg + 10))
        custom_svg += f"""
        <polygon points="{x1},{y1} {x2},{y2} {x3},{y3}" fill="black" />
        <text x="{x1+5}" y="{y1 + 5}" font-family="Arial" font-size="{int(16 * scale_factor)}" fill="black" text-anchor="end">{direction}</text>
        """
    
    # Draw the red triangle for the exact angle
    x1 = center + outer_radius * math.cos(math.radians(90 - angle))
    y1 = center - outer_radius * math.sin(math.radians(90 - angle))
    x2 = center + triangle_offset * math.cos(math.radians(80 - angle))
    y2 = center - triangle_offset * math.sin(math.radians(80 - angle))
    x3 = center + triangle_offset * math.cos(math.radians(100 - angle))
    y3 = center - triangle_offset * math.sin(math.radians(100 - angle))
    custom_svg += f"""
    <polygon points="{x1},{y1} {x2},{y2} {x3},{y3}" fill="red" />
    """

    custom_svg += """
    </svg>
    """
    return custom_svg

# Function to create a world map using Folium
def create_world_map(lat, lon):
    lat = convert_to_decimal_degrees(lat)
    lon = convert_to_decimal_degrees(lon)
    
    # Create the map with Folium
    m = folium.Map(location=[lat, lon], zoom_start=3)
    folium.Marker([lat, lon]).add_to(m)
    
    # Save the map to an HTML file
    map_html = m._repr_html_()
    
    # Update the map's HTML to adjust size
    custom_html = f"""
    <div style="width: 400px; height: 400px;">
        {map_html}
    </div>
    """
    return custom_html

# Function to create a gauge plot
def round_up_to_nearest(value, thresholds):
    for threshold in thresholds:
        if value <= threshold:
            return threshold
    return thresholds[-1]  # Return the largest threshold if value exceeds all thresholds

# Define thresholds
thresholds = [1, 5, 10, 50, 100, 500, 1000, 2000, 5000, 10000]

def create_gauge_plot(name, value, max_value):
    # Determine the nearest upper threshold
    rounded_max_value = round_up_to_nearest(max_value, thresholds)
    
    # Create the gauge plot
    custom_style = Style(
        value_font_size=50,  # Value font size
        label_font_size=20,  # Label font size
        major_label_font_size=20,  # Major label font size
        show_legend=False
    )
    
    gauge = pygal.SolidGauge(
        half_pie=True,
        inner_radius=0.70,
        style=custom_style,
        show_legend=False
    )
    
    # Add value and range to gauge
    gauge.title = name
    gauge.add(name, [{'value': value, 'max_value': rounded_max_value}], show_legend=False)
    
    return gauge.render_data_uri()

def create_engine_speed_line_graph(source):
    # Filter data by the selected source
    filtered_data = data[data['Source'] == source]
    
    # Filter for Engine Speed data
    engine_speed_data = filtered_data[filtered_data['Name'] == 'Engine Speed']
    
    # Prepare data for plotting
    timestamps = engine_speed_data['Timestamp'].tolist()
    speeds = engine_speed_data['Value'].astype(float).tolist()

    # Create line graph
    line_chart = pygal.Line(x_label_rotation=45, show_legend=False)
    line_chart.title = 'Engine Speed Over Time'
    line_chart.x_labels = [ts.strftime('%H:%M:%S') for ts in timestamps]
    line_chart.add('Engine Speed', speeds)

    return line_chart.render_data_uri()

def create_multi_parameter_line_graph(source, selected_parameter=None):
    # Filter data by the selected source
    filtered_data = data[data['Source'] == source]
    
    # Get unique parameters for the source
    unique_parameters = filtered_data['Name'].unique()
    
    # Create line graph
    line_chart = pygal.Line(x_label_rotation=45, show_legend=True)
    line_chart.title = 'Parameters Over Time'
    
    # Iterate over unique parameters
    for param in unique_parameters:
        param_data = filtered_data[filtered_data['Name'] == param]
        timestamps = param_data['Timestamp'].tolist()
        values = []

        # Convert values to float, skipping any non-numeric values
        for value in param_data['Value']:
            try:
                # Try to convert the value to float
                float_value = float(value)
                values.append(float_value)
            except ValueError:
                # Skip non-numeric values
                continue
        
        # Check if there are valid values to plot
        if len(values) > 0 and len(timestamps) == len(values):
            # Add to the graph if no parameter is selected or if it matches the selected parameter
            if selected_parameter is None or param == selected_parameter:
                line_chart.add(param, values)
    
    return line_chart.render_data_uri()

# Updated function to create warning buttons based on conditions
def update_warning_buttons(source, time):
    # Convert the time input to a datetime object
    try:
        selected_datetime = pd.to_datetime(time, format='%Y-%m-%d %H:%M:%S', errors='coerce')
        if pd.isna(selected_datetime):
            raise ValueError("Invalid timestamp format")
    except Exception as e:
        print(f"Error: {e}")
        return []
    
    # Real-time values for the selected source and time
    engine_speed = float(get_value_or_default(data[(data["Name"] == "Engine Speed") & (data["Source"] == source)], selected_datetime, source, "Engine Speed", 0))
    engine_temp = float(get_value_or_default(data[(data["Name"] == "Engine temp.") & (data["Source"] == source)], selected_datetime, source, "Engine temp.", 0))
    oil_pressure = float(get_value_or_default(data[(data["Name"] == "Engine oil pressure") & (data["Source"] == source)], selected_datetime, source, "Engine oil pressure", 0))
    fuel_level = float(get_value_or_default(data[(data["Name"] == "Fluid Level") & (data["Source"] == source)], selected_datetime, source, "Fluid Level", 0))
    
    # Maximum values in the dataset for each parameter
    max_engine_speed = float(data[(data["Name"] == "Engine Speed")]["Value"].max())
    max_engine_temp = float(data[(data["Name"] == "Engine temp.")]["Value"].max())
    max_oil_pressure = float(data[(data["Name"] == "Engine oil pressure")]["Value"].max())
    max_fuel_level = float(data[(data["Name"] == "Fluid Level")]["Value"].max())
    
    # Determine button colors based on the thresholds
    low_engine_speed_color = "danger" if engine_speed < 0.1 * max_engine_speed else ""
    high_engine_temp_color = "danger" if engine_temp > 0.9 * max_engine_temp else ""
    low_oil_pressure_color = "danger" if oil_pressure < 0.1 * max_oil_pressure else ""
    low_fuel_level_color = "danger" if fuel_level < 0.1 * max_fuel_level else ""
    
    # Create buttons
    warning_buttons = [
        widgets.Button(description=f'Low Engine Speed', button_style=low_engine_speed_color),
        widgets.Button(description=f'High Engine Temperature', button_style=high_engine_temp_color),
        widgets.Button(description=f'Low Oil Pressure', button_style=low_oil_pressure_color),
        widgets.Button(description=f'Low Fuel Level', button_style=low_fuel_level_color)
    ]
    
    return warning_buttons

# Function to get the value for the specified time and source or return default
def get_value_or_default(df, timestamp, source, name, default):
    filtered_df = df[(df['Timestamp'] == timestamp) & (df['Source'] == source) & (df['Name'] == name)]
    if not filtered_df.empty:
        value = filtered_df['Value'].iloc[0]
        if name in ["Latitude", "Longitude"]:
            return value  # Return the string value for latitude and longitude
        return float(value) if isinstance(value, (int, float, str)) else default
    return default

# Function to create numerical value displays
def create_numerical_values(source, time):
    # Get numerical values for the specified time and source
    speed_over_ground = get_value_or_default(data, time, source, "Speed Over Ground", 0)
    water_depth = get_value_or_default(data, time, source, "Water Depth", 0)
    tank_capacity = get_value_or_default(data, time, source, "Tank Capacity", 0)
    battery_voltage = get_value_or_default(data, time, source, "Battery Voltage", 0)

    # Create HTML for displaying the numerical values in rectangles
    numerical_values_html = f"""
    <div style="display: flex; justify-content: space-between; margin-bottom: 20px;">
        <div style="width: 22%; padding: 10px; background-color: #f0f0f0; border: 1px solid #ccc; text-align: center;">
            <strong>Speed Over Ground</strong><br>
            {speed_over_ground:.2f}
        </div>
        <div style="width: 22%; padding: 10px; background-color: #f0f0f0; border: 1px solid #ccc; text-align: center;">
            <strong>Water Depth</strong><br>
            {water_depth:.2f}
        </div>
        <div style="width: 22%; padding: 10px; background-color: #f0f0f0; border: 1px solid #ccc; text-align: center;">
            <strong>Tank Capacity</strong><br>
            {tank_capacity:.2f}
        </div>
        <div style="width: 22%; padding: 10px; background-color: #f0f0f0; border: 1px solid #ccc; text-align: center;">
            <strong>Battery Voltage</strong><br>
            {battery_voltage:.2f}
        </div>
    </div>
    """
    return numerical_values_html

# Widgets for user input
source_dropdown1 = widgets.Dropdown(
    options=all_sources,
    value=all_sources[0],
    description='Source:'
)

source_dropdown2 = widgets.Dropdown(
    options=all_sources,
    value=all_sources[0],
    description='Source:'
)

# Get unique parameters except Latitude, Longitude, and Deviation
parameters = [name for name in data['Name'].unique() if name not in ['Latitude', 'Longitude', 'Deviation']]
parameters.insert(0, 'All')

parameter_dropdown = widgets.Dropdown(
    options=parameters,
    value='All',
    description='Parameter:'
)

# Replace the SelectionSlider for time with a Text widget
time_input = widgets.Text(
    value=most_recent_time.strftime('%Y-%m-%d %H:%M:%S'),
    description='Time:',
    style={'description_width': 'initial'}  # Adjust description width to fit the label
)

# Create output widgets for each tab
output1 = widgets.Output()
output2 = widgets.Output()
output3 = widgets.Output()

# Function to update Tab 1
def update_tab1(source, time):
    with output1:
        output1.clear_output()  # Clear previous output
        
        # Validate and convert the time input
        try:
            time = pd.to_datetime(time, format='%Y-%m-%d %H:%M:%S', errors='coerce')
            if pd.isna(time):
                raise ValueError("Invalid timestamp format")
        except Exception as e:
            print(f"Error: {e}")
            return
        
        
        # Existing gauge charts
        engine_speed = get_value_or_default(data, time, source, "Engine Speed", 0)
        fuel_rate = get_value_or_default(data, time, source, "Fuel Rate", 0)
        
        # New gauge charts
        fluid_level = get_value_or_default(data, time, source, "Fluid Level", 0)
        trans_oil_temp = get_value_or_default(data, time, source, "Transmission oil temperature", 0)
        trans_oil_pressure = get_value_or_default(data, time, source, "Transmission oil pressure", 0)
        engine_tilt_trim = get_value_or_default(data, time, source, "Engine tilt/trim", 0)
        fuel_pressure = get_value_or_default(data, time, source, "Fuel Pressure", 0)
        engine_coolant_pressure = get_value_or_default(data, time, source, "Engine coolant pressure", 0)
        engine_temp = get_value_or_default(data, time, source, "Engine temp.", 0)
        engine_oil_temp = get_value_or_default(data, time, source, "Engine oil temp.", 0)
        engine_oil_pressure = get_value_or_default(data, time, source, "Engine oil pressure", 0)
        engine_boost_pressure = get_value_or_default(data, time, source, "Engine Boost Pressure", 0)

        # Display numerical values and gauge charts in rows (4 per row)
        numerical_values_html = create_numerical_values(source, time)
        gauge_objects = [
            f'<div style="display: inline-block; width: 225px; vertical-align: top;"><img src="{create_gauge_plot("Engine Speed", engine_speed, 2000)}" /></div>',
            f'<div style="display: inline-block; width: 225px; vertical-align: top;"><img src="{create_gauge_plot("Fuel Rate", fuel_rate, 1)}" /></div>',
            f'<div style="display: inline-block; width: 225px; vertical-align: top;"><img src="{create_gauge_plot("Fluid Level", fluid_level, 100)}" /></div>',
            f'<div style="display: inline-block; width: 225px; vertical-align: top;"><img src="{create_gauge_plot("Transmission Oil Temperature", trans_oil_temp, 500)}" /></div>',
            f'<div style="display: inline-block; width: 225px; vertical-align: top;"><img src="{create_gauge_plot("Transmission Oil Pressure", trans_oil_pressure, 1000000)}" /></div>',
            f'<div style="display: inline-block; width: 225px; vertical-align: top;"><img src="{create_gauge_plot("Engine Tilt/Trim", engine_tilt_trim, 100)}" /></div>',
            f'<div style="display: inline-block; width: 225px; vertical-align: top;"><img src="{create_gauge_plot("Fuel Pressure", fuel_pressure, 1000)}" /></div>',
            f'<div style="display: inline-block; width: 225px; vertical-align: top;"><img src="{create_gauge_plot("Engine Coolant Pressure", engine_coolant_pressure, 1000000)}" /></div>',
            f'<div style="display: inline-block; width: 225px; vertical-align: top;"><img src="{create_gauge_plot("Engine Temp.", engine_temp, 500)}" /></div>',
            f'<div style="display: inline-block; width: 225px; vertical-align: top;"><img src="{create_gauge_plot("Engine Oil Temp.", engine_oil_temp, 500)}" /></div>',
            f'<div style="display: inline-block; width: 225px; vertical-align: top;"><img src="{create_gauge_plot("Engine Oil Pressure", engine_oil_pressure, 1000000)}" /></div>',
            f'<div style="display: inline-block; width: 225px; vertical-align: top;"><img src="{create_gauge_plot("Engine Boost Pressure", engine_boost_pressure, 1000000)}" /></div>'
        ]
        
        gauge_html = ''.join(gauge_objects)
        display(HTML(f"{gauge_html}<div>{numerical_values_html}</div>"))
        
        warning_buttons = update_warning_buttons(source, time)
        display(widgets.HBox(warning_buttons))


# Function to update Tab 2
def update_tab2(source, time):
    with output2:
        output2.clear_output()  # Clear previous output
        
        # Validate and convert the time input
        try:
            time = pd.to_datetime(time, format='%Y-%m-%d %H:%M:%S', errors='coerce')
            if pd.isna(time):
                raise ValueError("Invalid timestamp format")
        except Exception as e:
            print(f"Error: {e}")
            return
        
        # Get values for all charts
        course_angle = get_value_or_default(cog_data, time, source, "Course Over Ground", 0)
        heading_angle = get_value_or_default(heading_data, time, source, "Heading", 0)
        latitude = get_value_or_default(data, time, source, "Latitude", "0° 0.0' N")
        longitude = get_value_or_default(data, time, source, "Longitude", "0° 0.0' E")
        

        # Display compass and geo map side by side
        side_by_side = widgets.HBox([
            widgets.HTML(value=create_compass(course_angle)),
            widgets.HTML(value=create_compass(heading_angle)),
            widgets.HTML(value=create_world_map(latitude, longitude))
            
        ])
        
        display(side_by_side)
        #display(widgets.HTML(value=create_compass(heading_angle)))
        
def update_tab3(source, parameter):
    with output3:
        output3.clear_output()  # Clear previous output
        
        # Create SVGs for the charts
        engine_speed_svg = create_engine_speed_line_graph(source)
        multi_param_svg = create_multi_parameter_line_graph(source, parameter if parameter != 'All' else None)
        
        # Display the charts side by side
        display(HTML(f"""
        <div style="display: flex; justify-content: space-between; align-items: flex-start;">
            <div style="flex: 1; margin-right: 5px;">
                <object type="image/svg+xml" data="{engine_speed_svg}" style="width: 100%; height: 400px; border: none;"></object>
            </div>
            <div style="flex: 1; margin-left: 5px;">
                <object type="image/svg+xml" data="{multi_param_svg}" style="width: 100%; height: 400px; border: none;"></object>
            </div>
        </div>
        """))

# Observers to update the outputs
source_dropdown1.observe(lambda change: update_tab1(source_dropdown1.value, time_input.value), names='value')
time_input.observe(lambda change: update_tab1(source_dropdown1.value, time_input.value), names='value')
source_dropdown1.observe(lambda change: update_tab2(source_dropdown1.value, time_input.value), names='value')
time_input.observe(lambda change: update_tab2(source_dropdown1.value, time_input.value), names='value')
source_dropdown2.observe(lambda change: update_tab3(source_dropdown2.value, parameter_dropdown.value), names='value')
parameter_dropdown.observe(lambda change: update_tab3(source_dropdown2.value, parameter_dropdown.value), names='value')

# Initial update for both tabs
update_tab1(all_sources[0], most_recent_time.strftime('%Y-%m-%d %H:%M:%S'))
update_tab2(all_sources[0], most_recent_time.strftime('%Y-%m-%d %H:%M:%S'))
update_tab3(all_sources[0], 'All')

# Create tabs
tab = widgets.Tab(children=[
    widgets.VBox([source_dropdown1, time_input, output1]), 
    widgets.VBox([source_dropdown1, time_input, output2]), 
    widgets.VBox([source_dropdown2, parameter_dropdown, output3])
])

tab.set_title(0, 'Tab 1')
tab.set_title(1, 'Tab 2')
tab.set_title(2, 'Tab 3')

# Display the tab
display(tab)

C:\Users\Rajasree Unnithan R\AppData\Local\Temp\ipykernel_37536\793476668.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cog_data['Value'] = cog_data['Value'].astype(float) * (180 / math.pi)
C:\Users\Rajasree Unnithan R\AppData\Local\Temp\ipykernel_37536\793476668.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  heading_data['Value'] = heading_data['Value'].astype(float) * (180 / math.pi)
